In [1]:
#download ok for 2014 & 2015
#2016 data needs curation

In [1]:
import pandas as pd

import dask
import dask.dataframe as dd
from dask.distributed import Client, progress, wait

from gcsfs import GCSFileSystem


In [2]:
gcs = GCSFileSystem(token='anon')
gcs.ls('anaconda-public-data/nyc-taxi/csv/2016')

['anaconda-public-data/nyc-taxi/csv/2016/yellow_tripdata_2016-01.csv',
 'anaconda-public-data/nyc-taxi/csv/2016/yellow_tripdata_2016-02.csv',
 'anaconda-public-data/nyc-taxi/csv/2016/yellow_tripdata_2016-03.csv',
 'anaconda-public-data/nyc-taxi/csv/2016/yellow_tripdata_2016-04.csv',
 'anaconda-public-data/nyc-taxi/csv/2016/yellow_tripdata_2016-05.csv',
 'anaconda-public-data/nyc-taxi/csv/2016/yellow_tripdata_2016-06.csv',
 'anaconda-public-data/nyc-taxi/csv/2016/yellow_tripdata_2016-07.csv',
 'anaconda-public-data/nyc-taxi/csv/2016/yellow_tripdata_2016-08.csv',
 'anaconda-public-data/nyc-taxi/csv/2016/yellow_tripdata_2016-09.csv',
 'anaconda-public-data/nyc-taxi/csv/2016/yellow_tripdata_2016-10.csv',
 'anaconda-public-data/nyc-taxi/csv/2016/yellow_tripdata_2016-11.csv',
 'anaconda-public-data/nyc-taxi/csv/2016/yellow_tripdata_2016-12.csv']

In [3]:
with gcs.open('anaconda-public-data/nyc-taxi/csv/2016/yellow_tripdata_2016-01.csv') as f:
    #df = pd.read_csv(f, nrows=5)                     
    df = pd.read_csv(f, nrows=5, parse_dates=['tpep_pickup_datetime', 'tpep_dropoff_datetime'])

df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,2,2016-01-01,2016-01-01,2,1.10,-73.990372,40.734695,1,N,-73.981842,40.732407,2,7.5,0.5,0.5,0,0,0.3,8.8
1,2,2016-01-01,2016-01-01,5,4.90,-73.980782,40.729912,1,N,-73.944473,40.716679,1,18.0,0.5,0.5,0,0,0.3,19.3
2,2,2016-01-01,2016-01-01,1,10.54,-73.984550,40.679565,1,N,-73.950272,40.788925,1,33.0,0.5,0.5,0,0,0.3,34.3
3,2,2016-01-01,2016-01-01,1,4.75,-73.993469,40.718990,1,N,-73.962242,40.657333,2,16.5,0.0,0.5,0,0,0.3,17.3
4,2,2016-01-01,2016-01-01,3,1.76,-73.960625,40.781330,1,N,-73.977264,40.758514,2,8.0,0.0,0.5,0,0,0.3,8.8


In [4]:
def pandas_to_parquet():
    for i in range(1, 1):
        fname = "yellow_tripdata_2015-%02d" % i
        fname_csv = "%s.csv" % fname
        fname_parquet = "%s.parquet" % fname

        print("downloading %s" % fname_csv)
        with gcs.open('anaconda-public-data/nyc-taxi/csv/2015/%s' % fname_csv) as f:
            df = pd.read_csv(f, parse_dates=['tpep_pickup_datetime', 'tpep_dropoff_datetime'])

        print("converting to parquet: %s" % fname_parquet)
        df.to_parquet("data/%s" % fname_parquet)

Parallelize Pandas with Dask.dataframe

In [5]:
client = Client()
client

<Client: 'tcp://172.18.0.4:8786' processes=4 threads=32, memory=251.25 GiB>

In [12]:
nyc_datatype = {'VendorID': 'string',
                'passenger_count': 'int32',
                'trip_distance': 'float32',
                'pickup_longitude': 'float32',
                'pickup_latitude': 'float32',
                'RateCodeID': 'string',
                'store_and_fwd_flag': 'string',
                'dropoff_longitude': 'float32',
                'dropoff_latitude': 'float32',
                'payment_type': 'string',
                'fare_amount': 'float32',
                'extra': 'float32',
                'mta_tax': 'float32',
                'tip_amount': 'float32',
                'tolls_amount': 'float32',
                'improvement_surcharge': 'float32',
                'total_amount':'float32' }

In [7]:
def data_2014_adjustment(df):
    #2014 data colums renaming
    colnames = []
    for c in df.columns:
        colnames += [c.replace(' ', '')]

    df.columns = colnames
    df = df.rename(columns={'pickup_datetime':'tpep_pickup_datetime',
                       'dropoff_datetime':'tpep_dropoff_datetime',
                       'rate_code':'RateCodeID',
                       'vendor_id':'VendorID',
                       'surcharge':'improvement_surcharge' })
    df["extra"] = 0

    df = df[['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
           'passenger_count', 'trip_distance', 'pickup_longitude',
           'pickup_latitude', 'RateCodeID', 'store_and_fwd_flag',
           'dropoff_longitude', 'dropoff_latitude', 'payment_type', 'fare_amount',
           'extra', 'mta_tax', 'tip_amount', 'tolls_amount',
           'improvement_surcharge', 'total_amount']]
    print(df.columns)
    return df

#2014 Data with columns adjustments

In [9]:
df = dd.read_csv('gcs://anaconda-public-data/nyc-taxi/csv/2014/yellow_*.csv',
                 storage_options={'token': 'anon'}, 
                 dtype={' tolls_amount': 'float64'},
                 parse_dates=[' pickup_datetime', ' dropoff_datetime'])
#.head(n=5)

In [10]:
df = data_2014_adjustment(df)
df = df.astype(nyc_datatype)

df = df.persist()
progress(df)
#wait(df)

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'pickup_longitude',
       'pickup_latitude', 'RateCodeID', 'store_and_fwd_flag',
       'dropoff_longitude', 'dropoff_latitude', 'payment_type', 'fare_amount',
       'extra', 'mta_tax', 'tip_amount', 'tolls_amount',
       'improvement_surcharge', 'total_amount'],
      dtype='object')


VBox()

In [11]:
%time print(df.passenger_count.sum().compute())

df.to_parquet('data/yellow_tripdata_2014.parquet', overwrite=True)

279997507
CPU times: user 32.5 ms, sys: 3.98 ms, total: 36.5 ms
Wall time: 286 ms


#2015 Data

In [16]:
df = dd.read_csv('gcs://anaconda-public-data/nyc-taxi/csv/2015/yellow_*.csv',
                 storage_options={'token': 'anon'}, 
                 parse_dates=['tpep_pickup_datetime', 'tpep_dropoff_datetime'])


In [17]:
df = df.persist()
progress(df)
#wait(df)

VBox()

In [18]:
%time print(df.passenger_count.sum().compute())
#df = df.astype(nyc_datatype)

df.to_parquet('data/yellow_tripdata_2015.parquet', overwrite=True)

245566747
CPU times: user 29.9 ms, sys: 1.34 ms, total: 31.3 ms
Wall time: 226 ms


#2016 Data

Data cleansing required, lot of errors

worker2_1    | Exception: 'ValueError(\'Mismatched dtypes found in `pd.read_csv`/`pd.read_table`.\\n\\n+---------------+--------+----------+\\n
| Column        | Found  | Expected |\\n
+---------------+--------+----------+\\n| VendorID      | object | int64    |\\n| trip_distance | object | float64  |\\n+---------------+--------+----------+\\n\\nThe following columns also raised exceptions on conversion:\\n\\n- VendorID\\n  ValueError("invalid literal for int() with base 10: \\\'2016-11-01 00:17:01\\\'")\\n- trip_distance\\n  ValueError("could not convert string to float: \\\'N\\\'")\\n\\nUsually this is due to dask\\\'s dtype inference failing, and\\n*may* be fixed by specifying dtypes manually by adding:\\n\\ndtype={\\\'VendorID\\\': \\\'object\\\',\\n       \\\'trip_distance\\\': \\\'object\\\'}\\n\\nto the call to `read_csv`/`read_table`.\\n\\n-------------------------------------------------------------\\n\\nThe following columns also failed to properly parse as dates:\\n\\n- tpep_pickup_datetime\\n- tpep_dropoff_datetime\\n\\nThis is usually due to an invalid value in that column. To\\ndiagnose and fix it\\\'s recommended to drop these columns from the\\n`parse_dates` keyword, and manually convert them to dates later\\nusing `dd.to_datetime`.\')'

In [47]:
df = dd.read_csv('gcs://anaconda-public-data/nyc-taxi/csv/2016/yellow_*.csv',
                 storage_options={'token': 'anon'}, 
                 dtype={'VendorID': 'int32',
                        'passenger_count': 'int32',
                        'trip_distance': 'string',
                        'pickup_longitude': 'float32',
                        'pickup_latitude': 'float32',
                        'RateCodeID': 'int64',
                        'store_and_fwd_flag': 'string',
                        'dropoff_longitude': 'float32',
                        'dropoff_latitude': 'float32',
                        'payment_type': 'float32',
                        'fare_amount': 'float32',
                        'extra': 'float32',
                        'mta_tax': 'float32',
                        'tip_amount': 'string',
                        'tolls_amount': 'float32',
                        'improvement_surcharge': 'float32',
                        'total_amount':'float32' },
                 parse_dates=['tpep_pickup_datetime', 'tpep_dropoff_datetime'])

In [48]:
df = df.persist()
progress(df)
#wait(df)

VBox()

In [18]:
df.dtypes

VendorID                         string
tpep_pickup_datetime     datetime64[ns]
tpep_dropoff_datetime    datetime64[ns]
passenger_count                   int32
trip_distance                   float32
pickup_longitude                float32
pickup_latitude                 float32
RateCodeID                       string
store_and_fwd_flag               string
dropoff_longitude               float32
dropoff_latitude                float32
payment_type                     string
fare_amount                     float32
extra                           float32
mta_tax                         float32
tip_amount                      float32
tolls_amount                    float32
improvement_surcharge           float32
total_amount                    float32
dtype: object

In [32]:
#%time print(df.passenger_count.sum().compute())
print(df.compute())

#df.to_parquet('data/yellow_tripdata_2016.parquet', overwrite=True)

                                  VendorID tpep_pickup_datetime  \
2 2016-12-31 15:15:01  2016-12-31 15:15:09                    1   
1 2016-12-01 00:00:01  2016-12-01 00:10:22                    1   
  2016-12-01 00:00:01  2016-12-01 00:11:01                    1   
2 2016-12-01 00:00:02  2016-12-01 00:09:17                    6   
  2016-12-01 00:00:02  2016-12-01 00:15:20                    1   
...                                    ...                  ...   
1 2016-12-31 23:59:54  2017-01-01 00:19:41                    1   
2 2016-12-31 23:59:54  2017-01-01 00:04:29                    1   
  2016-12-31 23:59:54  2017-01-01 00:09:26                    2   
  2016-12-31 23:59:54  2017-01-01 00:09:44                    1   
1 2016-12-31 23:59:58  2017-01-01 00:03:50                    1   

                      tpep_dropoff_datetime  passenger_count trip_distance  \
2 2016-12-31 15:15:01                   0.0                1             N   
1 2016-12-01 00:00:01                  